# Movie Plots Until 2017

In [8]:
import pandas as pd
movies_df = pd.read_csv('data/wiki_movie_plots_deduped.csv')
movies_df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [7]:
movies_df[movies_df['Release Year'] < 2017]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34877,2013,Particle (film),Turkish,Erdem Tepegöz,"Jale Arıkan, Rüçhan Caliskur, Özay Fecht, Remz...",drama film,https://en.wikipedia.org/wiki/Particle_(film),"Zeynep lost her job at weaving factory, and he..."
34878,2014,Mandıra Filozofu,Turkish,Director: Müfit Can Saçıntı,Director: Müfit Can Saçıntı\r\nCast: Rasim Özt...,unknown,https://en.wikipedia.org/wiki/Mand%C4%B1ra_Fil...,Cavit an ambitious industralist in İstanbul pl...
34879,2014,Winter Sleep,Turkish,Director: Nuri Bilge Ceylan,Director: Nuri Bilge Ceylan\r\nCast: Haluk Bil...,unknown,https://en.wikipedia.org/wiki/Winter_Sleep_(film),"Aydın, a former actor, owns a mountaintop hote..."
34880,2014,Sivas,Turkish,Director: Kaan Müjdeci,Director: Kaan Müjdeci\r\nCast: Dogan Izci,unknown,https://en.wikipedia.org/wiki/Sivas_(film),The film follows an eleven-year-old boy named ...


In [10]:
def filter_by_exact_word(df, column_name, search_word):
    """
    Filters the DataFrame by rows where the specified column contains the exact search word.

    Parameters:
    df (pd.DataFrame): The DataFrame to filter.
    column_name (str): The name of the column to search.
    search_word (str): The exact word to search for.

    Returns:
    pd.DataFrame: The filtered DataFrame.
    """
    # Construct the regular expression for exact word match
    search_pattern = r'\b' + search_word + r'\b'
    
    # Filter the DataFrame
    filtered_df = df[df[column_name].str.contains(search_pattern, case=False, na=False, regex=True)]
    
    return filtered_df

# Use the function to filter the DataFrame
search_word = 'Avengers'
result_df = filter_by_exact_word(movies_df, 'Title', search_word)

result_df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
5047,1950,The Avengers,American,John H. Auer,"Adele Mara, John Carroll",adventure,https://en.wikipedia.org/wiki/The_Avengers_(19...,"In 17th-century colonial Argentina, while tryi..."
13193,1998,The Avengers,American,Jeremiah S. Chechik,"Ralph Fiennes, Uma Thurman, Sean Connery, Jim ...",action,https://en.wikipedia.org/wiki/The_Avengers_(19...,The film opens with John Steed (Ralph Fiennes)...
16178,2012,"Avengers, TheThe Avengers",American,Joss Whedon,"Robert Downey, Jr., Chris Evans, Mark Ruffalo,...",superhero,https://en.wikipedia.org/wiki/The_Avengers_(20...,"The Asgardian Loki encounters the Other, the l..."
16983,2015,Avengers: Age of Ultron,American,Joss Whedon,"Robert Downey, Jr.\r\nChris Evans\r\nChris Hem...",action,https://en.wikipedia.org/wiki/Avengers:_Age_of...,"In the Eastern European country of Sokovia, th..."
17403,1973,Avengers of the Reef,Australian,Chris McCullough,"Simon Drake, Noel Ferrier, Garry McDonald, Jud...",adventure,https://en.wikipedia.org/wiki/Avengers_of_the_...,Scientists Bill Stewart goes to Fiji with his ...
23081,1978,Crippled Avengers,Hong Kong,Chang Cheh,NaN,unknown,https://en.wikipedia.org/wiki/Crippled_Avengers,"Chu Twin, a master of tiger style kung fu, ret..."
23133,1981,Masked Avengers,Hong Kong,Chang Cheh,"Chiang Sheng, Chu Ko, Chien Hsiao-hao",unknown,https://en.wikipedia.org/wiki/Masked_Avengers_...,The insidious Masked Gang of hired killers hav...


# IMDB Web Scraping BeautifulSoup

## Search

In [11]:
import requests
import re
from bs4 import BeautifulSoup

search_term = 'Thor'

# Escape special characters in the search term (not necessary here but good practice)
escaped_search_term = re.escape(search_term)

# Create a regex pattern to match the whole word "Thor"
pattern = rf'\b{escaped_search_term}\b'

# search_term = f'\b{search_term}\b'

# Define the URL
url = f'https://www.imdb.com/find/?q={search_term}'

# Define headers including a User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Make an HTTP request to the URL with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raise an exception for HTTP errors

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Print out the parsed HTML to inspect it (optional)
# print(soup.prettify())

# Find all <a> tags with the class 'ipc-metadata-list-summary-item__t'
links = soup.find_all('a', class_='ipc-metadata-list-summary-item__t')

# Extract titles and their corresponding title codes
titles_and_codes = []
for link in links:
    title = link.get_text()
    href = link.get('href')
    
    # Extract the title code from the href attribute
    if href and href.startswith('/title/'):
        title_code = href.split('/')[2]
    else:
        title_code = None
    
    titles_and_codes.append((title, title_code))

# Print the results
for title, code in titles_and_codes:
    print(f'Title: {title}, Code: {code}')

Title: Thor, Code: tt0800369
Title: Thor: Tag der Entscheidung, Code: tt3501632
Title: Thor: Love and Thunder, Code: tt10648342
Title: Thor: The Dark Kingdom, Code: tt1981115
Title: Thor - Der unbesiegbare Barbar, Code: tt0200211
Title: London Thor, Code: None
Title: Thora Birch, Code: None
Title: Cameron Thor, Code: None
Title: Thor, Code: None
Title: Thor Knai, Code: None


## Title Display

In [12]:
import requests
from bs4 import BeautifulSoup


title_code = 'tt0200211'

# Define the URL (use the actual URL if you want to fetch live data)
url = f'https://www.imdb.com/title/{title_code}/'

# Define headers including a User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Make an HTTP request to the URL with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raise an exception for HTTP errors

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the title from the <title> tag
title_tag = soup.find('title')
title = title_tag.get_text() if title_tag else 'No title found'

# Extract the description from the <meta> tag with name='description'
meta_description_tag = soup.find('meta', attrs={'name': 'description'})
description = meta_description_tag.get('content') if meta_description_tag else 'No description found'

# Print the extracted title and description
print('Title:', title)
print('Description:', description)


Title: Thor - Der unbesiegbare Barbar (1983) - IMDb
Description: Thor - Der unbesiegbare Barbar: Directed by Tonino Ricci. With Bruno Minniti, Maria Romano, Malisa Longo, Raf Baldassarre. In the vein of CONAN THE BARBARIAN and Lucio Fulci's CONQUEST comes a tale of mythology and magic, of how THOR, a legendary god, triumphs over overwhelming odds to great victory and the destruction of his foes. After both his parents are brutally murdered by his father's rival Gnut and his men the new born Thor is placed in hiding by the physical embodiment of the god Teisha. Raised in secret under the guidance of Teisha, Thor comes to maturity and goes on a quest to avenge the death of his parents and return peace to his lands, in the process he discovers a woman to take as his wife. Taking the beautiful warrior virgin, Ina, as his companion, his exploits lead to a confrontation with Gnut, the slayer of his father. He finds and uncovers his father's sword and masters the art of combat, all under the 

## Wikipedia API

In [23]:
import requests
from bs4 import BeautifulSoup

def get_film_details(title):
    base_url = "https://en.wikipedia.org/wiki/"
    url = base_url + title.replace(" ", "_")
    
    # Fetch the content from the Wikipedia page
    response = requests.get(url)
    if response.status_code != 200:
        return "Failed to retrieve page."

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract the infobox
    infobox = soup.find('table', {'class': 'infobox'})
    if not infobox:
        return "Infobox not found."

    # Extract data from the infobox
    details = {}
    for row in infobox.find_all('tr'):
        header = row.find('th')
        data = row.find('td')
        if header and data:
            header_text = header.get_text(strip=True)
            data_text = data.get_text(strip=True)
            details[header_text] = data_text

    # Extract the plot section
    # Find the "Plot" section
    plot_section = soup.find('h2', id='Plot')

    # Extract text from the following <p> tags
    plot = ''
    for sibling in plot_section.find_all_next(['p']):
        if sibling.name == 'div' and sibling.find('h2'):
            break
        plot += sibling.get_text()

        # Extract actors section
        actors = "Not Available"
        cast_section = soup.find('span', {'id': 'Cast'})
        if cast_section:
            cast_content = cast_section.find_next('ul')
            if cast_content:
                actors = ', '.join([li.get_text(strip=True) for li in cast_content.find_all('li')])

    # Extract year and genre
    year = details.get('Release date', 'Not Available')
    genre = details.get('Genre', 'Not Available')

    return (f"Title: {title}\n\n"
            f"Extract:\n{details.get('Plot', 'Not Available')}\n\n"
            f"Plot:\n{plot}\n\n"
            f"Actors:\n{actors}\n\n"
            f"Year:\n{year}\n\n"
            f"Genre:\n{genre}")

# Example usage
film_title = "Thor:_Love_and_Thunder"  # Replace with the film you are interested in
print(get_film_details(film_title))


Title: Thor:_Love_and_Thunder

Extract:
Not Available

Plot:
Gorr and his daughter, Love, the last of their race, struggle in a barren desert. Despite their prayers to their god, Rapu, Love dies. The god-killing Necrosword calls to Gorr, leading him to Rapu's lush realm. After Rapu cruelly mocks and dismisses Gorr's plight, he renounces the god, causing Rapu to start strangling him. The Necrosword offers itself to Gorr, who kills Rapu with it and vows to kill all gods. Gorr is granted the ability to manipulate shadows and produce monsters but is cursed with impending death and corruption under the sword's influence.
Thor has joined the Guardians of the Galaxy after the Avengers' battle against Thanos.[a] He learns of a distress signal from Sif, so he parts ways with the team. He finds an injured Sif, who warns that Gorr's next target is New Asgard. Meanwhile, Dr. Jane Foster, Thor's ex-girlfriend, has been diagnosed with stage four terminal cancer. With medical treatment proving ineffe

In [34]:
from bs4 import BeautifulSoup

# Parse the HTML
# soup = BeautifulSoup(html_content, 'html.parser')

title = "The_Godfather" 

base_url = "https://en.wikipedia.org/wiki/"
url = base_url + title.replace(" ", "_")
    
    # Fetch the content from the Wikipedia page
response = requests.get(url)
if response.status_code != 200:
    print("Failed to retrieve page.")    

soup = BeautifulSoup(response.content, 'html.parser')

# Find the table with the class "infobox vevent"
table = soup.find('table', class_='infobox vevent')

# Extract all table rows
rows = table.find_all('tr')

# Extract content from each row and store in separate variables
row_data = [row.get_text(separator=' ', strip=True) for row in rows]

# Print each row content
for i, row in enumerate(row_data):
    print(f"Row {i+1}: {row}")


Row 1: The Godfather
Row 2: Theatrical release poster
Row 3: Directed by Francis Ford Coppola
Row 4: Screenplay by Mario Puzo Francis Ford Coppola
Row 5: Based on The Godfather by Mario Puzo
Row 6: Produced by Albert S. Ruddy
Row 7: Starring Marlon Brando Al Pacino James Caan Richard Castellano Robert Duvall Sterling Hayden John Marley Richard Conte Diane Keaton
Row 8: Cinematography Gordon Willis
Row 9: Edited by William Reynolds Peter Zinner
Row 10: Music by Nino Rota
Row 11: Production companies Paramount Pictures Alfran Productions
Row 12: Distributed by Paramount Pictures
Row 13: Release dates March 14, 1972 ( 1972-03-14 ) ( Loew's State Theatre ) March 24, 1972 ( 1972-03-24 ) (United States)
Row 14: Running time 175 minutes [ 1 ]
Row 15: Country United States
Row 16: Language English
Row 17: Budget $6–7.2 million [ N 1 ]
Row 18: Box office $250–291 million [ N 2 ]


In [37]:
# Find the table with the class "infobox vevent"
table = soup.find('table', class_='infobox vevent')

# Extract all paragraphs that follow this table
paragraphs_after_table = []
for sibling in table.find_all_next():
    if sibling.name == 'p':
        paragraphs_after_table.append(sibling.get_text(strip=True))
    elif sibling.name == 'div' and sibling.find('h2'):
        # Assuming you want to stop collecting paragraphs if another section starts
        break

# Print the paragraphs
for i, paragraph in enumerate(paragraphs_after_table):
    print(f"Paragraph {i+1}: {paragraph} ")

Paragraph 1: The Godfatheris a 1972 Americanepicgangster film[2]directed byFrancis Ford Coppola, who co-wrote the screenplay withMario Puzo, based on Puzo's best-selling1969 novel. The film stars anensemble castincludingMarlon Brando,Al Pacino,James Caan,Richard Castellano,Robert Duvall,Sterling Hayden,John Marley,Richard Conte, andDiane Keaton. It is the first installment inThe Godfathertrilogy,  chronicling theCorleone familyunder patriarchVito Corleone(Brando) from 1945 to 1955. It focuses on the transformation of his youngest son,Michael Corleone(Pacino), from reluctant family outsider to ruthlessmafia boss. 
Paragraph 2: Paramount Picturesobtained the rights to the novel for $80,000, before it gained popularity.[3][4]Studio executives had trouble finding a director; the first few candidates turned down the position before Coppola signed on to direct the film but disagreement followed over casting several characters, in particular, Vito (Marlon Brando) and Michael (Al Pacino). Film

In [38]:
paragraphs_after_table

["The Godfatheris a 1972 Americanepicgangster film[2]directed byFrancis Ford Coppola, who co-wrote the screenplay withMario Puzo, based on Puzo's best-selling1969 novel. The film stars anensemble castincludingMarlon Brando,Al Pacino,James Caan,Richard Castellano,Robert Duvall,Sterling Hayden,John Marley,Richard Conte, andDiane Keaton. It is the first installment inThe Godfathertrilogy,  chronicling theCorleone familyunder patriarchVito Corleone(Brando) from 1945 to 1955. It focuses on the transformation of his youngest son,Michael Corleone(Pacino), from reluctant family outsider to ruthlessmafia boss.",
 'Paramount Picturesobtained the rights to the novel for $80,000, before it gained popularity.[3][4]Studio executives had trouble finding a director; the first few candidates turned down the position before Coppola signed on to direct the film but disagreement followed over casting several characters, in particular, Vito (Marlon Brando) and Michael (Al Pacino). Filming took place primar